In [ ]:
#@Author: Roufa

'''
check if toc are in sentence case or title case.
ip:pdf
op: check whether TOC are consistent in style or not. we will go with dominant style as conistent style
and return non-domiant style bboxes.
'''

import sys
sys.path.append('/data/copy_assessment_tool/modules_final/')

import pdfplumber
from collections import Counter

from programmatic.TOC.extract_TOC import TOC_num
from utils import *
from title_sentence_case_consistency import *
from programmatic.json_output import return_json_result




def process_page_content(page_content_dict):
    new_dict = {}

    for key, value in page_content_dict.items():
        parts = key.split('~')
        chapter_title = parts[2]
        page_number = float(value)
        page_and_line = f"{parts[0]},{parts[1]}"
        new_dict[chapter_title] = (page_number, page_and_line)

    return new_dict


def coordinate_approach_new(coordinate_dict, unique_cords):
    '''This create the dicitonary with there respective keys based on coordinate'''
    predefined_chapter_map = ["Part", "Chapters", "sections", "subsections", "Headings", "others"]

    cord_map = {}
    for groupname, cord in zip(predefined_chapter_map, unique_cords):
        cord_map[cord] = groupname

    result_map = {'Part': {}, 'Chapters': {},"sections":{}}
    
    for text, cord in coordinate_dict.items():
        group_name = cord_map[cord[0]]
        if group_name == "Part":
            result_map['Part'][text] = [(int(cord[1].split(',')[0]), int(cord[1].split(',')[1]))]
        elif group_name == "Chapters":
            result_map['Chapters'][text] = [(int(cord[1].split(',')[0]), int(cord[1].split(',')[1]))]
        elif group_name == "sections":
            result_map['sections'][text] = [(int(cord[1].split(',')[0]), int(cord[1].split(',')[1]))]
        
    result_map = {key : val for key,val in result_map.items() if val}


    return result_map

def process_chapters(new_cleaned_data):
    '''This will extract coordinate or indentation values of each line of text in Toc'''
    predefined_chapter_map = ["Part", "Chapters", "sections", "subsections", "Titles","others"]
    all_cords = [cord[0] for cord in new_cleaned_data.values()]
    unique_cords = sorted(list(set(all_cords)))
    counter = Counter(all_cords)

    cord_map = {}
    for groupname, cord in zip(predefined_chapter_map, unique_cords):
        cord_map[cord] = groupname

    return cord_map, unique_cords

def check_if_part_in_chapter_new(result_map):
    
    ''' This function is used to segregate the toc based on if it starts with chapter,section and part it 
    will place in chapter  respective keys in dictionary '''
    preprocessed_data = {
    'chapters': {},
    'parts': {},
    'sections': {}
    }

    for key, value in result_map.items():    
        x = len(result_map)
  
        # intendation only have two coordinate value then check for the parts and chapter in dict
        if x <=2:
        
            for item_k,item_val in value.items():
                new_item = item_k[:]
                if item_k.lower().startswith(('chapter',"Chapter")):
                    preprocessed_data['chapters'][new_item] = item_val
                elif item_k.lower().startswith('part'):
                    preprocessed_data['parts'][new_item] = item_val
                elif item_k.lower().startswith('sections'):
                    preprocessed_data['parts'][new_item] = item_val
                else:
                    preprocessed_data['sections'][new_item] = item_val
    # preprocessed_cleaned = {key: value for key, value in preprocessed_data.items() if value}
    preprocessed_data = {key : val for key,val in preprocessed_data.items() if val}

    return preprocessed_data

def check_number_sequencing_new(preprocessed_cleaned_dict):
    '''this funciton used to check for number sequence of the chapters or sections '''
    
    new_dictionary = {"chapters":{},"sections":{},"part":{}}
    for key,values in preprocessed_cleaned_dict.items():
        for val_k,val_v in values.items():
            result_split = val_k.split('.')
        
            if len(result_split) > 1:
                if val_k.startswith(("part","Part")):
                    new_dictionary["part"][val_k] = val_v
                
                elif len(result_split) > 1:
                    if (result_split[0].strip().isdigit()) and (result_split[1].strip().split(' ')[0].isdigit()):
                        new_dictionary["sections"][val_k] = val_v
                    elif (result_split[0].strip().isdigit()) and (result_split[1].strip().split(' ')[0].isalpha()):
                        new_dictionary["chapters"][val_k] = val_v
            else:
                space_spliting = val_k.split(' ')[:2]  
                if len(space_spliting)>1:
                    if (space_spliting[0].strip().isdigit()) and (space_spliting[1].strip().split(' ')[0].isalpha()):
                        new_dictionary["chapters"][val_k] = val_v
                    else:
                        if val_k.startswith(("part","Part")):
                            new_dictionary["part"][val_k] = val_v
                        elif val_k.startswith(("chapter","Chapter")):
                            new_dictionary["chapters"][val_k] = val_v
                        else:
                            new_dictionary["sections"][val_k] = val_v
              
    new_dictionary= {key : val for key,val in new_dictionary.items() if val}
    return new_dictionary


def check_final_call_new(cleaned_data, unique_cords):
    coo = coordinate_approach_new(cleaned_data, unique_cords)
    coo = {key: value for key, value in coo.items() if value}
    
    if len(coo) <= 2:
        cp = check_if_part_in_chapter_new(coo)
        
        coo.clear()
        cp = {key: value for key, value in cp.items() if value}
        
        if (len(cp) <= 1) and ("sections" in cp):
            seq = check_number_sequencing_new(cp)
            cp.clear()
            seq = {key: value for key, value in seq.items() if value}
        else:
            seq = {}  # Assign a default value
    else:
        coo = coo
        cp = {}  # Assign a default value
        seq = {}
    return coo, cp, seq


def dict_list_contents(dict_list):
    # This functins is to take input as list of dictionary which is return from check_final_call
    add_diction = [] 
    for d in dict_list:
        for d_k,d_v in d.items():
            if d_k:
                add_diction.append(d)
            else:
                print("No contens are extracted",d)
    return add_diction[0]


def final_chapter_call(pdf):

    entity_labels = ["person"]
    entries_to_remove = ['Contents', 'Foreword', 'Preface', 'List of figures', 'Acknowledgment', 'List of contributors', 'List of abbreviations', 'Conventions', 'List of tables', 'About the Authors', 'References', 'appendix', 'index', 'Conclusions', "Devil Is An Ass", "Bibliography"]

    page_content_dict, _ = TOC_num(pdf) 
    new_dict_Toc= process_page_content(page_content_dict)
    filtered_page_content=remove_entries(new_dict_Toc,entries_to_remove) 
    new_cleaned_data = filter_entries_by_named_entities(filtered_page_content,entity_labels)
    cord_map, unique_cords = process_chapters(new_cleaned_data)
    coo, cp, seq= check_final_call_new(new_cleaned_data, unique_cords)
    dict_list= [coo, cp, seq]
    final_dict_toc= dict_list_contents(dict_list)
    cc_Clean = remove_numbers_Toc_new(final_dict_toc)
    return cc_Clean


def check_TOC_title_sent_case_inconsistency(pdf):
    inconsistent_TOC_count = 0

    cc_Clean = final_chapter_call(pdf)
    chapters, sections, _, parts = process_text_data(cc_Clean)

    print(chapters)
    print()
    print(sections)
    print()
    print(parts)
    print()
    count_non_title_case_chapters, non_title_case_chapters, count_non_title_case_parts, non_title_case_parts, inconsistent_sections = get_inconsisent_TOC(chapters, parts, sections)

    #get inconsistent section counts and details
    inconsistent_sec_counts = 0
    inconsistent_sec_details = [{}]
    if inconsistent_sections:
        inconsistent_sec = [my_dict[key]  for my_dict in inconsistent_sections for key in my_dict if key.startswith('count')]
        inconsistent_sec_counts = sum(inconsistent_sec)

        #inconsistent_sec_details = [my_dict[key] for my_dict in inconsistent_sections for key in my_dict if key.endswith('details')]
        for my_dict in inconsistent_sections:
            for key, value in my_dict.items():
                if key.endswith('details'):
                    inconsistent_sec_details[0].update(value)

    inconsistent_TOC_count = count_non_title_case_chapters + count_non_title_case_parts + inconsistent_sec_counts
    return inconsistent_TOC_count, non_title_case_chapters, non_title_case_parts, inconsistent_sec_details




def get_inconsistent_TOC_bboxes(file_path):
    results = []
    pdf = pdfplumber.open(file_path)
    inconsistent_TOC_count, non_title_case_chapters, non_title_case_parts, inconsistent_sec_details = check_TOC_title_sent_case_inconsistency(pdf)

    inconsistent_sec_details += [non_title_case_chapters, non_title_case_parts]
    results,length = final_consistent_text_json(pdf,inconsistent_sec_details)
    
    inconsistent_TOC_bboxes = return_json_result(results)
    return inconsistent_TOC_count, inconsistent_TOC_bboxes

# file_path = '/data/copy_assessment_tool/modules/data/15032-5196-FullBook.pdf'
file_path ='/data/copy_assessment_tool/modules/data/15031-4988-FullBook.pdf'




get_inconsistent_TOC_bboxes(file_path)